## Problématique 4 – Avoir joué dans un club augmente-t-il les chances d’y entraîner ?

### Objectif :

Nous cherchons à savoir si un coach ayant été joueur d’un club augmente ses chances d’en devenir plus tard l’entraîneur.

Plutôt que de comparer tous les clubs indistinctement, nous nous concentrons ici sur les **20 clubs de Serie A lors de la saison 2024-2025**.  
Pour chaque coach, et pour chacun de ces clubs :

- A-t-il **joué** dans ce club au cours de sa carrière ?
- A-t-il **entraîné** ce club ?

Cela permet d’établir un **tableau croisé** (2x2) :

|               | A coaché le club | N’a pas coaché |
|---------------|------------------|----------------|
| A joué        | ?                | ?              |
| N’a pas joué  | ?                | ?              |

Nous utiliserons un **test du Chi²** pour vérifier s’il existe une **relation significative** entre le fait d’avoir joué dans un club et le fait de l’avoir entraîné plus tard.


### Requête SQL utilisée

Pour obtenir les informations nécessaires à notre analyse, nous avons effectué la requête suivante sur notre base de données SQLite :

```sql
WITH serieA_2024_teams AS (
  SELECT id FROM teams
  WHERE name IN (
    'SSC Napoli', 'Inter Milan', 'Juventus FC', 'Atalanta BC', 'A.S. Roma', 'ACF Fiorentina', 'SS Lazio', 'AC Milan',
    'Bologna F.C. 1909', 'Torino FC', 'Como 1907', 'Udinese Calcio', 'Genoa CFC', 'Hellas Verona FC',
    'Cagliari Calcio', 'Parma Calcio 1913', 'US Lecce', 'Empoli FC', 'Venezia F.C.', 'AC Monza'
  )
),
played AS (
  SELECT DISTINCT coach_id, team_id FROM player_mandates
  WHERE team_id IN (SELECT id FROM serieA_2024_teams)
),
coached AS (
  SELECT DISTINCT coach_id, team_id FROM mandates
  WHERE team_id IN (SELECT id FROM serieA_2024_teams)
)
SELECT
  CASE WHEN p.coach_id IS NOT NULL THEN 1 ELSE 0 END AS played_there,
  CASE WHEN c.coach_id IS NOT NULL THEN 1 ELSE 0 END AS coached_there,
  COUNT(*) AS count
FROM (
  SELECT coach_id, team_id FROM played
  UNION
  SELECT coach_id, team_id FROM coached
) AS all_pairs
LEFT JOIN played p ON all_pairs.coach_id = p.coach_id AND all_pairs.team_id = p.team_id
LEFT JOIN coached c ON all_pairs.coach_id = c.coach_id AND all_pairs.team_id = c.team_id
GROUP BY played_there, coached_there;


### Résultats de la requête

Voici le tableau de contingence obtenu :

| A joué | A coaché | Nombre d’observations |
|--------|----------|------------------------|
| Non    | Non      | 4483                   |
| Non    | Oui      | 234                    |
| Oui    | Non      | 403                    |
| Oui    | Oui      | 93                     |

Ce tableau est la base de notre test du **Chi²**, afin d’évaluer si avoir été joueur dans un club est associé de manière significative au fait d’y devenir entraîneur.


In [16]:
import pandas as pd
import scipy.stats as stats

# Tableau de contingence
data = [[93, 403], [234, 4483]]
df = pd.DataFrame(data, columns=["Coaché", "Pas coaché"], index=["Joué", "Pas joué"])

# Chi²
chi2, p, dof, expected = stats.chi2_contingency(df)

print(" Statistique du test :", round(chi2, 2))
print(" p-value :", round(p, 4))
print(" Degrés de liberté :", dof)
print("\n Tableau des valeurs attendues :\n", pd.DataFrame(expected, columns=["Coaché", "Pas coaché"], index=["Joué", "Pas joué"]))

 Statistique du test : 142.81
 p-value : 0.0
 Degrés de liberté : 1

 Tableau des valeurs attendues :
               Coaché   Pas coaché
Joué       31.112987   464.887013
Pas joué  295.887013  4421.112987


### Test du Chi²

Nous avons appliqué un test du Chi² pour évaluer l’indépendance entre les deux variables suivantes :

- **A joué dans le club** (oui/non)
- **A coaché le club** (oui/non)

#### Résultats du test :

- **Statistique du test :** 142.81
- **p-value :** < 0.0001
- **Degrés de liberté :** 1

#### Interprétation :

La p-value extrêmement faible (< 0.0001) indique que nous pouvons **rejeter l’hypothèse d’indépendance** entre le fait d’avoir joué dans un club et celui de l’avoir entraîné.

Autrement dit, **les coachs sont significativement plus susceptibles de devenir entraîneurs dans un club où ils ont déjà joué.**

 **Limites de l’analyse :**
- Nous ne considérons ici que les coachs de clubs de Serie A en 2024-2025
- Nous n’avons pas de données complètes sur les joueurs **qui n’ont jamais entraîné**
- Ce résultat montre une corrélation, **pas une causalité**
